In [117]:
using LowLevelFEM

In [118]:
function square_mesh(; x0=0.0, y0=0.0, lx=1.0, ly=1.0, n=10, dx=lx / n, dy=ly / n, order=1)

    gmsh.option.setNumber("General.Verbosity", 0)
    gmsh.model.add("square")
    # --------------------------------------------------
    # Geometry
    # --------------------------------------------------
    # Box: origin (0,0,0), size l x l x l
    box = gmsh.model.occ.addRectangle(x0, y0, 0.0, lx, ly)

    gmsh.model.occ.synchronize()
    #return

    # --------------------------------------------------
    # Physical groups
    # --------------------------------------------------
    gmsh.model.addPhysicalGroup(1, [1], -1, "bottom")
    gmsh.model.addPhysicalGroup(1, [2], -1, "right")
    gmsh.model.addPhysicalGroup(1, [3], -1, "top")
    gmsh.model.addPhysicalGroup(1, [4], -1, "left")

    gmsh.model.addPhysicalGroup(2, [1], -1, "surface")

    gmsh.model.addPhysicalGroup(0, [1], -1, "leftbottom")
    gmsh.model.addPhysicalGroup(0, [2], -1, "rightbottom")
    gmsh.model.addPhysicalGroup(0, [3], -1, "righttop")
    gmsh.model.addPhysicalGroup(0, [4], -1, "lefttop")

    # --------------------------------------------------
    # Mesh settings (structured hex mesh)
    # --------------------------------------------------
    #gmsh.option.setNumber("Mesh.Algorithm3D", 1)      # Delaunay for recombination
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    #gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 1)

    #gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

    d = min(dx, dy)

    gmsh.model.mesh.setTransfiniteCurve(1, ceil(lx / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(3, ceil(lx / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(2, ceil(ly / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(4, ceil(ly / d) + 1)

    gmsh.model.mesh.setTransfiniteSurface(1)

    gmsh.option.setNumber("Mesh.ElementOrder", order)

    # Characteristic length control
    #lc = min(dx, dy)
    #gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
    #gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

    # --------------------------------------------------
    # Generate mesh
    # --------------------------------------------------
    gmsh.model.mesh.generate(2)

    # --------------------------------------------------
    # Save
    # --------------------------------------------------
    #gmsh.write("cube.msh")

    #gmsh.finalize()

    return nothing
end


square_mesh (generic function with 1 method)

In [119]:
"""
    airyLoadVector(problem, loads)

Assembles the boundary load vector for the Airy stress function Φ
based on the weak form term:

    ∫_Γ t · ∇v

Valid for 2D scalar problems (Φ).

Arguments:
- problem :: Problem   (2D, pdim = 1)
- loads   :: Vector{BoundaryCondition}
            using fx, fy as traction components

Return:
- ScalarField
"""
function airyLoadVector(problem::Problem, loads::Vector{BoundaryCondition})

    @assert problem.dim == 2 "Airy load only implemented for 2D."
    @assert problem.pdim == 1 "Airy Φ must be scalar (pdim = 1)."

    gmsh.model.setCurrent(problem.name)

    non = problem.non
    fp = zeros(non, 1)

    # nodal coordinates
    nodeTags, nodeCoord, _ = gmsh.model.mesh.getNodes()
    ncoord = reshape(nodeCoord, 3, :)

    for bc in loads

        name = bc.phName
        txF = bc.fx !== nothing ? bc.fx : 0.0
        tyF = bc.fy !== nothing ? bc.fy : 0.0

        dimTags = gmsh.model.getEntitiesForPhysicalName(name)

        for (dim, tag) in dimTags

            @assert dim == 1 "Airy traction must be applied on boundary curves (dim=1)."

            elementTypes, elementTags, elemNodeTags =
                gmsh.model.mesh.getElements(dim, tag)

            for it in eachindex(elementTypes)

                et = elementTypes[it]
                conn = elemNodeTags[it]
                elementName, edim, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                # Gauss rule
                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))
                numIntPoints = length(intWeights)

                # reference gradients
                comp, fun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")

                gradRef = reshape(fun, comp, numNodes, numIntPoints)

                for e in 1:length(elementTags[it])

                    elem = elementTags[it][e]

                    # connectivity
                    nodes = conn[(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)

                    Jac = reshape(jac, 3, :)

                    for j in 1:numIntPoints

                        # 2x2 Jacobian
                        #Jmat = Jac[1:2, 2*j-1:2*j]
                        #invJT = inv(Jmat)'

                        # Gauss point coords
                        x = 0.0
                        y = 0.0
                        for a in 1:numNodes
                            Na = 1.0 / numNodes  # only for coord interpolation if needed
                            x += ncoord[1, nodes[a]] * Na
                            y += ncoord[2, nodes[a]] * Na
                        end

                        # evaluate traction
                        tx = txF isa Function ? txF(x, y, 0.0) : txF
                        ty = tyF isa Function ? tyF(x, y, 0.0) : tyF

                        # boundary Jacobian (curve length scale)
                        Ja = sqrt(Jac[1, 2*j-1]^2 + Jac[2, 2*j-1]^2)

                        for a in 1:numNodes
                            #gradGlob = invJT * gradRef[1:2, a, j]
                            dNdx = gradRef[1, a, j]
                            dNdy = gradRef[2, a, j]

                            fp[nodes[a]] +=
                                (tx * dNdx + ty * dNdy) *
                                Ja *
                                intWeights[j]
                        end
                    end
                end
            end
        end
    end

    return ScalarField([], fp, [0], [], 1, :scalar, problem)
end

airyLoadVector

In [120]:
using LinearAlgebra, SparseArrays

In [121]:
"""
    airyLoadVector2D(problem, loads; surface_physical_groups=nothing)

Assemble the boundary load vector for Airy-type weak form contribution

    ∫_Γ t · ∇v

for a 2D scalar field (Φ or ψ).

This implementation is mesh-agnostic: it integrates over boundary (dim=1)
elements but evaluates ∇N using the adjacent 2D (dim=2) parent element.

Assumptions / scope:
- Works for 2D meshes consisting of triangles and/or quadrangles (any order).
- `loads` contains BoundaryCondition with `phName` and components `fx`,`fy`
  as Number or Function (x,y,z)->value.

Returns:
- ScalarField with size (non × 1).
"""
function airyLoadVector2D(problem::Problem, loads::Vector{BoundaryCondition}; surface_physical_groups=nothing)
    @assert problem.dim == 2 "airyLoadVector2D: only for 2D problems."
    @assert problem.pdim == 1 "airyLoadVector2D: scalar field expected (pdim=1)."

    gmsh.model.setCurrent(problem.name)
    println("Entering airyLoadVector2D")

    # --------------------------
    # Node coordinates lookup
    # --------------------------
    nodeTags, nodeCoord, _ = gmsh.model.mesh.getNodes()
    # nodeCoord is flat [x1,y1,z1,x2,y2,z2,...]
    ncoord = reshape(nodeCoord, 3, :)

    non = problem.non
    fp = zeros(non, 1)

    # --------------------------
    # Build edge -> (parent elem tag, parent elemType, sideId)
    # --------------------------
    # We hash edges by sorted pair of *primary* corner node tags.
    #
    # For quadratic/higher order elements: edge has mid-side nodes too,
    # but the two end vertices uniquely identify the edge, and that's enough
    # to find the parent and which side we are on.
    #
    # sideId convention:
    #  - Triangle: 1:(1-2), 2:(2-3), 3:(3-1)
    #  - Quad:     1:(1-2), 2:(2-3), 3:(3-4), 4:(4-1)
    #
    edge2parent = Dict{Tuple{Int,Int},Tuple{Int,Int,Int}}()  # (n1,n2)->(elemTag, elemType, sideId)

    function _edgekey(a, b)
        return a < b ? (a, b) : (b, a)
    end

    function _corner_nodes(elemType, nodes)
        # Use gmsh element properties to find how many primary nodes (corner nodes).
        _, _, _, numNodes, _, numPrimaryNodes = gmsh.model.mesh.getElementProperties(elemType)
        # For standard Lagrange triangles/quads, primary nodes are the vertices first.
        return nodes[1:numPrimaryNodes], numPrimaryNodes
    end

    function _add_surface_edges!(elemType, elemTag, nodes)
        corners, npc = _corner_nodes(elemType, nodes)

        # Identify triangle vs quad from num primary nodes
        if npc == 3
            # triangle corners: 1-2,2-3,3-1
            e = ((_edgekey(corners[1], corners[2]), 1),
                (_edgekey(corners[2], corners[3]), 2),
                (_edgekey(corners[3], corners[1]), 3))
            for (k, sid) in e
                # store first one; if non-manifold boundary, you might want a vector here
                edge2parent[k] = (elemTag, elemType, sid)
            end
        elseif npc == 4
            # quad corners: 1-2,2-3,3-4,4-1
            e = ((_edgekey(corners[1], corners[2]), 1),
                (_edgekey(corners[2], corners[3]), 2),
                (_edgekey(corners[3], corners[4]), 3),
                (_edgekey(corners[4], corners[1]), 4))
            for (k, sid) in e
                edge2parent[k] = (elemTag, elemType, sid)
            end
        else
            # If you later add other 2D element types, extend here.
            return
        end
    end

    # Collect surface elements either from provided physical groups or from all dim=2 entities
    dim2_entities = surface_physical_groups === nothing ?
                    gmsh.model.getEntities(2) :
                    vcat([gmsh.model.getEntitiesForPhysicalName(g) for g in surface_physical_groups]...)

    println("11111111111111111")
    for (dim, tag) in dim2_entities
        @assert dim == 2
        elementTypes, elementTags, elemNodeTags = gmsh.model.mesh.getElements(dim, tag)
        for it in eachindex(elementTypes)
            et = elementTypes[it]
            # getElementProperties tells us how many nodes per element
            _, _, _, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)
            conn = elemNodeTags[it]
            nelem = length(elementTags[it])
            @inbounds for e in 1:nelem
                elemTag = elementTags[it][e]
                nodes = conn[(e-1)*numNodes+1:e*numNodes]
                _add_surface_edges!(et, elemTag, nodes)
            end
        end
    end

    println("2222222222222222222")
    # --------------------------
    # Helper: map 1D edge Gauss parameter -> parent (ξ,η) in gmsh reference
    # --------------------------
    # We will use the edge integration points provided by gmsh for the *edge element*.
    # For standard Lagrange line element, the reference coordinate is u in [-1,1].
    #
    # Triangle reference in gmsh for basis functions is typically (u,v) with u>=0,v>=0,u+v<=1
    # but gmsh integration points for triangles are given in that convention.
    #
    # Practical robust trick:
    #   - For TRI: we map u_edge ∈ [-1,1] -> s ∈ [0,1] via s=(u_edge+1)/2
    #   - Then define (u,v) on the triangle edge by side:
    #       side 1 (1-2): (u,v) = (s, 0)
    #       side 2 (2-3): (u,v) = (1-s, s)
    #       side 3 (3-1): (u,v) = (0, 1-s)
    #
    # For QUAD: gmsh reference is (u,v) ∈ [-1,1]×[-1,1]
    #   - use u_edge directly as s ∈ [-1,1]
    #   - side mapping:
    #       1 (1-2): (u,v)=(s,-1)
    #       2 (2-3): (u,v)=( 1,s)
    #       3 (3-4): (u,v)=(s, 1)
    #       4 (4-1): (u,v)=(-1,s)
    #
    # This is the only place where "triangle vs quad" matters.
    function _edge_to_parent_uv(npc, sideId, uedge)
        if npc == 3
            s = 0.5 * (uedge + 1.0) # [-1,1] -> [0,1]
            if sideId == 1
                return (s, 0.0)
            elseif sideId == 2
                return (1.0 - s, s)
            else
                return (0.0, 1.0 - s)
            end
        elseif npc == 4
            s = uedge # already in [-1,1]
            if sideId == 1
                return (s, -1.0)
            elseif sideId == 2
                return (1.0, s)
            elseif sideId == 3
                return (s, 1.0)
            else
                return (-1.0, s)
            end
        else
            error("airyLoadVector2D: unsupported element (numPrimaryNodes=$npc).")
        end
    end

    println("3333333333333333")
    # --------------------------
    # Loop loads and assemble
    # --------------------------
    for bc in loads
        name = bc.phName
        txF = bc.fx !== nothing ? bc.fx : 0.0
        tyF = bc.fy !== nothing ? bc.fy : 0.0

        dimTags = gmsh.model.getEntitiesForPhysicalName(name)
        for (dim, tag) in dimTags
            println("44444444444444")
            @assert dim == 1 "airyLoadVector2D: loads must be applied on boundary curves (dim=1)."

            edgeTypes, edgeTags, edgeNodeTags = gmsh.model.mesh.getElements(dim, tag)

            for it in eachindex(edgeTypes)
                et_edge = edgeTypes[it]
                _, _, order_edge, numNodes_edge, _, numPrimary_edge = gmsh.model.mesh.getElementProperties(et_edge)
                conn_edge = edgeNodeTags[it]
                ne_edge = length(edgeTags[it])

                # 1D Gauss points on the edge element
                intPoints1D, intWeights1D = gmsh.model.mesh.getIntegrationPoints(et_edge, "Gauss" * string(2order_edge + 1))
                nGP = length(intWeights1D)

                # Edge basis for geometry evaluation (to get x,y at GP)
                compN, funN, _ = gmsh.model.mesh.getBasisFunctions(et_edge, intPoints1D, "Lagrange")
                Nedge = reshape(funN, numNodes_edge, nGP)

                println("it=", it)
                @inbounds for e in 1:ne_edge
                    edTag = edgeTags[it][e]
                    nodes_edge = conn_edge[(e-1)*numNodes_edge+1:e*numNodes_edge]

                    # Build edge key from end vertices (primary nodes)
                    n1 = nodes_edge[1]
                    n2 = nodes_edge[2]
                    k = _edgekey(n1, n2)

                    haskey(edge2parent, k) || continue
                    parentTag, parentType, sideId = edge2parent[k]

                    # Parent element connectivity needed for evaluation
                    _, _, _, numNodes_parent, _, numPrimary_parent = gmsh.model.mesh.getElementProperties(parentType)
                    # We can get parent element node tags via getElements... but we already have mapping only tag.
                    # gmsh can give element node tags for a given element tag:
                    parentNodes = gmsh.model.mesh.getElement(parentTag)[3]  # returns (type, tags, nodeTags); [3] is nodeTags
                    parentNodes = collect(parentNodes) # ensure Vector{Int}

                    println("e=", e)
                    # Loop GP on edge
                    for j in 1:nGP
                        uedge = intPoints1D[j]  # for 1D, intPoints is [u1,u2,...]
                        u, v = _edge_to_parent_uv(numPrimary_parent, sideId, uedge)

                        # Parent basis and grad basis at (u,v)
                        # localCoord is concatenated (u,v,w); w=0 for 2D
                        localCoord = [u, v, 0.0]

                        # Geometry: x,y from parent Lagrange
                        _, funP, _ = gmsh.model.mesh.getBasisFunctions(parentType, localCoord, "Lagrange")
                        Np = funP[1:numNodes_parent]  # scalar basis values

                        x = 0.0
                        y = 0.0
                        for a in 1:numNodes_parent
                            x += Np[a] * ncoord[1, parentNodes[a]]
                            y += Np[a] * ncoord[2, parentNodes[a]]
                        end

                        tx = txF isa Function ? txF(x, y, 0.0) : txF
                        ty = tyF isa Function ? tyF(x, y, 0.0) : tyF
                        println("Parent tag: ", parentTag)
                        println("localCoord: ", localCoord)
                        # Parent Jacobian at (u,v) to get inv(J)^T for gradients + edge length scale
                        jac, _, _ = gmsh.model.mesh.getJacobian(parentTag, localCoord)
                        # jac is flat 3x? but for 2D element we take first 2 rows and first 2 columns
                        Jac = reshape(jac, 3, :)
                        Jmat = Jac[1:2, 1:2]
                        invJT = inv(Jmat)'

                        # Parent GradLagrange in reference coords
                        compG, funG, _ = gmsh.model.mesh.getBasisFunctions(parentType, localCoord, "GradLagrange")
                        # funG layout: compG*numNodes_parent
                        gradRef = reshape(funG, compG, numNodes_parent)  # compG is typically 3
                        # use only first 2 ref components
                        # grad_x = inv(J)^T * grad_ξ
                        # Edge tangent for ds scaling:
                        # For TRI: our parameter s changes along one direction; for QUAD similar.
                        # We can compute physical tangent by differentiating x(u,v) along uedge direction.
                        #
                        # Easier robust option: use the EDGE element Jacobian for |dx/ds|:
                        jacE, _, _ = gmsh.model.mesh.getJacobian(edTag, [uedge])
                        JacE = reshape(jacE, 3, :)
                        Ja = sqrt(JacE[1, 1]^2 + JacE[2, 1]^2)  # |dx/du_edge|
                        w = intWeights1D[j]

                        for a in 1:numNodes_parent
                            gref = gradRef[1:2, a]                 # (∂N/∂u, ∂N/∂v)
                            gxy = invJT * gref                     # (∂N/∂x, ∂N/∂y)

                            fp[parentNodes[a], 1] += (tx * gxy[1] + ty * gxy[2]) * Ja * w
                        end
                    end
                end
            end
        end
    end

    return ScalarField([], fp, [0], [], 1, :scalar, problem)
end

airyLoadVector2D

In [122]:
"""
    airyLoadVector2D_stable(problem, loads; surface_physical_groups=["surface"])

Assemble scalar RHS for the weak term:
    ∫_Γ t · ∇v  dΓ
for 2D scalar fields (Airy Φ or ψ).

- Integrates on boundary edges (physical groups from `loads`)
- Evaluates ∇N from the adjacent 2D (surface) elements
- Works for triangles and quads (any polynomial order in gmsh Lagrange family)

`fx, fy` in BoundaryCondition may be Number or Function (x,y,z)->value.

Returns ScalarField (non×1).
"""
function airyLoadVector2D_stable(problem::Problem, loads::Vector{BoundaryCondition};
    surface_physical_groups=["surface"])

    @assert problem.dim == 2
    @assert problem.pdim == 1

    gmsh.model.setCurrent(problem.name)

    # --- node coordinates
    nodeTags, nodeCoord, _ = gmsh.model.mesh.getNodes()
    ncoord = reshape(nodeCoord, 3, :)  # 3×N

    non = problem.non
    fp = zeros(non, 1)

    # --- helper: edge key from two vertex node tags
    edgekey(a::Integer, b::Integer) = (Int(min(a, b)), Int(max(a, b)))

    # ----------------------------
    # 1) Collect boundary edges (as vertex pairs) per load group
    # ----------------------------
    # Map: edge (v1,v2) -> (txF, tyF)
    # If multiple BCs touch same edge, they sum naturally.
    edgeLoad = Dict{Tuple{Int,Int},Tuple{Any,Any}}()

    for bc in loads
        name = bc.phName
        txF = bc.fx !== nothing ? bc.fx : 0.0
        tyF = bc.fy !== nothing ? bc.fy : 0.0

        for (dim, tag) in gmsh.model.getEntitiesForPhysicalName(name)
            @assert dim == 1 "Airy traction must be on boundary curves (dim=1)."
            etypes, etags, enodes = gmsh.model.mesh.getElements(dim, tag)
            for it in eachindex(etypes)
                et = etypes[it]
                _, _, _, numNodes, _, numPrimary = gmsh.model.mesh.getElementProperties(et)
                conn = enodes[it]
                ne = length(etags[it])
                for e in 1:ne
                    nodes = conn[(e-1)*numNodes+1:e*numNodes]
                    v1 = nodes[1]
                    v2 = nodes[2]  # endpoints are primary for line elements
                    k = edgekey(v1, v2)
                    if haskey(edgeLoad, k)
                        # accumulate if needed (simple: wrap as sum functions/numbers)
                        txOld, tyOld = edgeLoad[k]
                        edgeLoad[k] = ((x, y, z) -> ((txOld isa Function ? txOld(x, y, z) : txOld) +
                                                     (txF isa Function ? txF(x, y, z) : txF)),
                            (x, y, z) -> ((tyOld isa Function ? tyOld(x, y, z) : tyOld) +
                                          (tyF isa Function ? tyF(x, y, z) : tyF)))
                    else
                        edgeLoad[k] = (txF, tyF)
                    end
                end
            end
        end
    end

    isempty(edgeLoad) && return ScalarField([], fp, [0], [], 1, :scalar, problem)

    # ----------------------------
    # 2) Surface entities to iterate
    # ----------------------------
    if surface_physical_groups isa String
        surface_physical_groups = [surface_physical_groups]
    end

    surfEnt = Tuple{Int,Int}[]
    for g in surface_physical_groups
        append!(surfEnt, gmsh.model.getEntitiesForPhysicalName(g))
    end

    # ----------------------------
    # 3) Reference mappings for element sides (tri/quad)
    # ----------------------------
    # 1D Gauss points on [-1,1] for edge integration will be taken from a line element type
    # BUT to avoid mixing element types, we just ask gmsh for integration points of a generic line:
    # use element type 1 (2-node line) if available; safer: take from existing boundary line type later.
    # Here: we will generate our own Gauss points using gmsh by calling getIntegrationPoints on a 2-node line.
    lineType = 1  # gmsh element type id for 2-node line
    _, _, orderL, _, _, _ = gmsh.model.mesh.getElementProperties(lineType)
    sPts, sWts = gmsh.model.mesh.getIntegrationPoints(lineType, "Gauss" * string(2 * orderL + 1))
    nGP = length(sWts)
    sPts = collect(sPts)  # length nGP

    # side maps return:
    # localCoordEdge :: Vector{Float64} of length 3*nGP  (u,v,w concatenated)
    # dudS, dvdS      :: constants per side (since our mapping is linear in s)
    function tri_side_points(side::Int, s::Vector{Float64})
        # s in [-1,1], map to r in [0,1]
        r = 0.5 .* (s .+ 1.0)
        u = similar(r)
        v = similar(r)
        if side == 1       # (1-2): v=0, u=r
            u .= r
            v .= 0
            dudS = 0.5
            dvdS = 0.0
        elseif side == 2   # (2-3): u=1-r, v=r
            u .= 1 .- r
            v .= r
            dudS = -0.5
            dvdS = 0.5
        else               # side 3 (3-1): u=0, v=1-r
            u .= 0
            v .= 1 .- r
            dudS = 0.0
            dvdS = -0.5
        end
        w = zeros(nGP)
        # concatenate as [u1,v1,w1, u2,v2,w2, ...]
        lc = Vector{Float64}(undef, 3nGP)
        @inbounds for i in 1:nGP
            lc[3i-2] = u[i]
            lc[3i-1] = v[i]
            lc[3i] = 0.0
        end
        return lc, dudS, dvdS
    end

    function quad_side_points(side::Int, s::Vector{Float64})
        u = similar(s)
        v = similar(s)
        if side == 1       # v=-1, u=s
            u .= s
            v .= -1
            dudS = 1.0
            dvdS = 0.0
        elseif side == 2   # u=+1, v=s
            u .= 1
            v .= s
            dudS = 0.0
            dvdS = 1.0
        elseif side == 3   # v=+1, u=s
            u .= s
            v .= 1
            dudS = 1.0
            dvdS = 0.0
        else               # side == 4: u=-1, v=s
            u .= -1
            v .= s
            dudS = 0.0
            dvdS = 1.0
        end
        lc = Vector{Float64}(undef, 3nGP)
        @inbounds for i in 1:nGP
            lc[3i-2] = u[i]
            lc[3i-1] = v[i]
            lc[3i] = 0.0
        end
        return lc, dudS, dvdS
    end

    # ----------------------------
    # 4) Loop surface elements, detect loaded sides, integrate
    # ----------------------------
    for (dim, tag) in surfEnt
        @assert dim == 2
        etypes, etags, enodes = gmsh.model.mesh.getElements(dim, tag)

        for it in eachindex(etypes)
            et = etypes[it]
            _, _, _, numNodes, _, numPrimary = gmsh.model.mesh.getElementProperties(et)
            conn = enodes[it]
            ne = length(etags[it])

            # side count and endpoint indices for corner nodes
            if numPrimary == 3
                sideCount = 3
                cornerPairs = ((1, 2), (2, 3), (3, 1))
                sidePoints = tri_side_points
            elseif numPrimary == 4
                sideCount = 4
                cornerPairs = ((1, 2), (2, 3), (3, 4), (4, 1))
                sidePoints = quad_side_points
            else
                continue
            end

            for e in 1:ne
                elem = etags[it][e]
                nodes = conn[(e-1)*numNodes+1:e*numNodes]
                # pre-cast to Int indices for array access
                nodesI = Int.(nodes)

                # Check which sides are in edgeLoad
                for side in 1:sideCount
                    a, b = cornerPairs[side]
                    k = edgekey(nodesI[a], nodesI[b])
                    haskey(edgeLoad, k) || continue
                    txF, tyF = edgeLoad[k]

                    # Build local coords for this side at nGP points
                    lc, dudS, dvdS = sidePoints(side, sPts)

                    # Basis on parent element at edge points
                    _, funN, _ = gmsh.model.mesh.getBasisFunctions(et, lc, "Lagrange")
                    Np = reshape(funN, :, nGP)  # numNodes × nGP

                    # Reference gradients on parent element
                    compG, funG, _ = gmsh.model.mesh.getBasisFunctions(et, lc, "GradLagrange")
                    gradRef = reshape(funG, compG, numNodes, nGP)  # compG×numNodes×nGP

                    # Jacobian on parent element at edge points (vectorized call!)
                    jac, _, _ = gmsh.model.mesh.getJacobian(elem, lc)
                    Jac = reshape(jac, 3, :)  # 3×(2*nGP) for 2D elements

                    @inbounds for j in 1:nGP
                        # physical point (x,y) from Np
                        x = 0.0
                        y = 0.0
                        for aN in 1:numNodes
                            x += Np[aN, j] * ncoord[1, nodesI[aN]]
                            y += Np[aN, j] * ncoord[2, nodesI[aN]]
                        end

                        tx = txF isa Function ? txF(x, y, 0.0) : txF
                        ty = tyF isa Function ? tyF(x, y, 0.0) : tyF

                        # Build 2×2 J = [x_u x_v; y_u y_v] from Jac columns
                        xu = Jac[1, 2*j-1]
                        xv = Jac[1, 2*j]
                        yu = Jac[2, 2*j-1]
                        yv = Jac[2, 2*j]
                        detJ = xu * yv - xv * yu
                        if !(abs(detJ) > 1e-14)
                            # skip degenerate point instead of crashing kernel
                            continue
                        end
                        invJT = (1 / detJ) * [yv -yu; -xv xu]  # inv(J)' without forming inv()

                        # tangent = x_u * du/ds + x_v * dv/ds (same for y)
                        txs = xu * dudS + xv * dvdS
                        tys = yu * dudS + yv * dvdS
                        Ja = sqrt(txs^2 + tys^2)   # |dx/ds| in xy
                        w = sWts[j]

                        # accumulate (t · gradN_xy) * Ja * w
                        for aN in 1:numNodes
                            gref = gradRef[1:2, aN, j]   # (∂N/∂u, ∂N/∂v)
                            gxy = invJT * gref          # (∂N/∂x, ∂N/∂y)
                            fp[nodesI[aN], 1] += (tx * gxy[1] + ty * gxy[2]) * Ja * w
                        end
                    end
                end
            end
        end
    end

    return ScalarField([], fp, [0], [], 1, :scalar, problem)
end

airyLoadVector2D_stable

In [123]:
gmsh.initialize()
square_mesh(order=4)

In [124]:
mat = Material("surface")
prob = Problem([mat], type=:ScalarField, dim=2)

Problem("square", :ScalarField, 2, 1, Material[Material("surface", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 1681, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [125]:
tractionBCR = BoundaryCondition("right", fx=1)
tractionBCL = BoundaryCondition("left", fx=-1)

BoundaryCondition("left", nothing, nothing, nothing, nothing, nothing, nothing, -1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [126]:
K = poissonMatrix(prob)
#fψ = airyLoadVector(prob, [tractionBCL, tractionBCR])

fψ = airyLoadVector2D_stable(prob, [tractionBCL, tractionBCR], surface_physical_groups=["surface"])


ψ = solveField(K, fψ, support=[BoundaryCondition("bottom", p=0.0), BoundaryCondition("top", p=0.0)])

K = poissonMatrix(prob)
fΦ = loadVector(prob, [BoundaryCondition("surface", q=-ψ)])

Φ = solveField(K, fΦ, support=[BoundaryCondition("leftbottom", p=0), BoundaryCondition("rightbottom", p=0), BoundaryCondition("lefttop", p=0)])

nodal ScalarField
[0.0; 0.0; … ; -3.6228740769677056e-6; 1.3217802736460446e-6;;]

In [127]:
showDoFResults(Φ, name="Φ")
showDoFResults(∂y(∂y(Φ)), name="σ x")
showDoFResults(∂x(∂x(Φ)), name="σ y")
showDoFResults(-∂x(∂y(Φ)), name="τ xy")

3

In [128]:
showDoFResults(fψ, name="fψ")
showDoFResults(fΦ, name="fΦ")

5

In [129]:
openPostProcessor()

In [130]:
gmsh.finalize()